# Filter female wavs

In [16]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import scipy 
import pandas as pd
import shutil

In [3]:
df=pd.read_csv(r"cremad\VideoDemographics.csv")
df.head(3)

,ActorID,Age,Sex,Race,Ethnicity
0,1001,51,Male,Caucasian,Not Hispanic
1,1002,21,Female,Caucasian,Not Hispanic
2,1003,21,Female,Caucasian,Not Hispanic


In [4]:
female_ids=[str(df.loc[i].iat[0]) for i in df.index if df.loc[i].iat[2]=="Female"]

In [5]:
audio_folder_name=r'cremad\AudioWAV'
dest_folder_name= r'cremad\FemaleAudioWAV'
file_names = listdir(audio_folder_name)
for f in file_names:
    
    x=f.split("_")
    if x[0] not in female_ids:
       continue
    #print(f)
    shutil.move(audio_folder_name + '\\' +f, dest_folder_name)
    

In [5]:
df_fem = df[df.Sex =='Female']
df_fem.head(5)

,ActorID,Age,Sex,Race,Ethnicity
1,1002,21,Female,Caucasian,Not Hispanic
2,1003,21,Female,Caucasian,Not Hispanic
3,1004,42,Female,Caucasian,Not Hispanic
5,1006,58,Female,Caucasian,Not Hispanic
6,1007,38,Female,African American,Not Hispanic


In [11]:
final_list= r'C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\FemaleAudioWAV'
file_names = listdir(final_list)
df2 = pd.DataFrame(file_names,columns=['Filename'])
df2.head()

,Filename
0,1002_DFA_ANG_XX.wav
1,1002_DFA_DIS_XX.wav
2,1002_DFA_FEA_XX.wav
3,1002_DFA_HAP_XX.wav
4,1002_DFA_NEU_XX.wav


In [18]:
def getEmotion(x):
    part = x.split('_')
    if part[2] == 'SAD':
        return 'sad'
    elif part[2] == 'ANG':
        return 'angry'
    elif part[2] == 'DIS':
        return 'disgust'
    elif part[2] == 'FEA':
        return 'fear'
    elif part[2] == 'HAP':
        return 'happy'
    elif part[2] == 'NEU':
        return 'neutral'
    else:
        return 'Unknown'
    
df2['Emotion'] = df2['Filename'].apply(getEmotion)


df2.Emotion.value_counts()

angry      600
disgust    600
happy      600
sad        600
fear       600
neutral    512
Name: Emotion, dtype: int64

In [23]:
def getLength(filename):
    audio_path = final_list + '\\' + filename
    data, sr = librosa.load(audio_path, sr = 22050)
    return np.size(data,0)/22050

df2['Length'] = df2['Filename'].apply(getLength)

df2.head(5)

,Filename,Emotion,Length
0,1002_DFA_ANG_XX.wav,angry,2.569252
1,1002_DFA_DIS_XX.wav,disgust,2.535918
2,1002_DFA_FEA_XX.wav,fear,2.736100
3,1002_DFA_HAP_XX.wav,happy,2.202222
4,1002_DFA_NEU_XX.wav,neutral,2.502540


In [25]:
df2.groupby(['Emotion']).mean()

,Length
Emotion,
angry,2.592484
disgust,2.812258
fear,2.551843
happy,2.365023
neutral,2.481414
sad,2.629143


In [26]:
df2.groupby(['Emotion']).min()

,Filename,Length
Emotion,,
angry,1002_DFA_ANG_XX.wav,1.501542
disgust,1002_DFA_DIS_XX.wav,1.568254
fear,1002_DFA_FEA_XX.wav,1.501542
happy,1002_DFA_HAP_XX.wav,1.334694
neutral,1002_DFA_NEU_XX.wav,1.401406
sad,1002_DFA_SAD_XX.wav,1.468163


In [27]:
df2.groupby(['Emotion']).max()

,Filename,Length
Emotion,,
angry,1091_WSI_ANG_XX.wav,4.971655
disgust,1091_WSI_DIS_XX.wav,4.871565
fear,1091_WSI_FEA_XX.wav,4.537914
happy,1091_WSI_HAP_XX.wav,3.903900
neutral,1091_WSI_NEU_XX.wav,4.771474
sad,1091_WSI_SAD_XX.wav,5.005034


# Convert to mel specs

In [19]:
##FUNCTIONS
def plot_signals(signals):
    fig, axes = plt.subplots(nrows=2, ncols=5, sharex=False,
                             sharey=True, figsize=(20,5))
    fig.suptitle('Time Series', size=16)
    i = 0
    for x in range(1):
        for y in range(1):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1
            
def convert_single_audio_to_image(audio_path, image_path, spectrogram_dimensions = (64,64)):
    data, sr = librosa.load(audio_path, sr = 22050)
    signals ={1:data}
    #plot_signals(signals)
    print("sr={}".format(sr))
    #Make a mel spectrogram from audio
    window_width =  0.025 #25 ms  window size 
    sliding = 0.01 #10ms stride 
    print("Length of audio:", np.size(data,0)/22050)
    spec = librosa.feature.melspectrogram(y=data, sr=sr, n_fft = int(window_width*sr), hop_length =int(sliding*sr))
    print(spec.shape)
    
    #Convert amplitude to decibels
    db_spec = librosa.power_to_db(spec, ref=np.max)

    #fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
    #Iterate over time in 100 ms steps, produce mel-specs with 100ms time
    fig = plt.figure()
    ax = plt.axes()
    ax.set_axis_off()
    middle = int(db_spec.shape[1]/2) #we want to get the sample from the middle of the signal for 1.3 s
    total_time = 130 #1.3 seconds
    half_time = int(total_time/2)
    #print('Middle', middle,'Half',half_time)
    librosa.display.specshow(np.reshape(np.copy(db_spec[:,middle-half_time:middle+half_time]),(128,-1)), sr=sr, hop_length =int(sliding*sr))
    fig.savefig(image_path,bbox_inches='tight', transparent=True, pad_inches=0.0)
    plt.close(fig)



In [20]:
if __name__ == '__main__':
    audio_folder_name= r"cremad\FemaleAudioWAV"
    image_folder_name=  r"cremad\SpectogramInitial"
    file_names = listdir(audio_folder_name)
#     file_names = [f for f in listdir(audio_folder_name)]
    print("count of Wav_files in {}= {}".format(len(file_names),audio_folder_name))
    for file_name in file_names:
        name, ext = os.path.splitext(file_name)
        print(file_name)
            
        audio_path = audio_folder_name + '\\' + file_name
        spectogram_path = image_folder_name + '\\' + file_name.replace('.wav', '.png')
        convert_single_audio_to_image(audio_path,spectogram_path)
    image_files= listdir(image_folder_name)
    print("count of Wav_files in {}= {}".format(len(image_files),image_folder_name))

count of Wav_files in 3512= cremad\FemaleAudioWAV
1002_DFA_ANG_XX.wav
sr=22050
Length of audio: 2.569251700680272
(128, 258)
1002_DFA_DIS_XX.wav
sr=22050
Length of audio: 2.5359183673469388
(128, 255)
1002_DFA_FEA_XX.wav
sr=22050
Length of audio: 2.73609977324263
(128, 275)
1002_DFA_HAP_XX.wav
sr=22050
Length of audio: 2.2022222222222223
(128, 221)
1002_DFA_NEU_XX.wav
sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1002_DFA_SAD_XX.wav
sr=22050
Length of audio: 2.3690249433106576
(128, 238)
1002_IEO_ANG_HI.wav
sr=22050
Length of audio: 2.73609977324263
(128, 275)
1002_IEO_ANG_LO.wav
sr=22050
Length of audio: 2.4024036281179137
(128, 241)
1002_IEO_ANG_MD.wav
sr=22050
Length of audio: 2.769478458049887
(128, 278)
1002_IEO_DIS_HI.wav
sr=22050
Length of audio: 3.0030385487528344
(128, 301)
1002_IEO_DIS_LO.wav
sr=22050
Length of audio: 2.8695691609977323
(128, 288)
1002_IEO_DIS_MD.wav
sr=22050
Length of audio: 2.6026303854875286
(128, 261)
1002_IEO_FEA_HI.wav
sr=22050
Length of audio

sr=22050
Length of audio: 2.836190476190476
(128, 285)
1003_ITH_ANG_XX.wav
sr=22050
Length of audio: 3.670340136054422
(128, 368)
1003_ITH_DIS_XX.wav
sr=22050
Length of audio: 4.037414965986395
(128, 405)
1003_ITH_FEA_XX.wav
sr=22050
Length of audio: 3.5369160997732427
(128, 355)
1003_ITH_HAP_XX.wav
sr=22050
Length of audio: 3.1364625850340135
(128, 315)
1003_ITH_NEU_XX.wav
sr=22050
Length of audio: 4.037414965986395
(128, 405)
1003_ITH_SAD_XX.wav
sr=22050
Length of audio: 3.6369614512471657
(128, 365)
1003_ITS_ANG_XX.wav
sr=22050
Length of audio: 3.737097505668934
(128, 375)
1003_ITS_DIS_XX.wav
sr=22050
Length of audio: 3.703718820861678
(128, 372)
1003_ITS_FEA_XX.wav
sr=22050
Length of audio: 3.1364625850340135
(128, 315)
1003_ITS_HAP_XX.wav
sr=22050
Length of audio: 3.737097505668934
(128, 375)
1003_ITS_NEU_XX.wav
sr=22050
Length of audio: 3.737097505668934
(128, 375)
1003_ITS_SAD_XX.wav
sr=22050
Length of audio: 3.5035374149659866
(128, 352)
1003_IWL_ANG_XX.wav
sr=22050
Length of a

sr=22050
Length of audio: 3.2032199546485263
(128, 322)
1004_MTI_HAP_XX.wav
sr=22050
Length of audio: 3.2032199546485263
(128, 322)
1004_MTI_NEU_XX.wav
sr=22050
Length of audio: 3.703718820861678
(128, 372)
1004_MTI_SAD_XX.wav
sr=22050
Length of audio: 3.3033560090702947
(128, 332)
1004_TAI_ANG_XX.wav
sr=22050
Length of audio: 3.3700226757369616
(128, 338)
1004_TAI_DIS_XX.wav
sr=22050
Length of audio: 3.9039002267573695
(128, 392)
1004_TAI_FEA_XX.wav
sr=22050
Length of audio: 2.936281179138322
(128, 295)
1004_TAI_HAP_XX.wav
sr=22050
Length of audio: 2.6026303854875286
(128, 261)
1004_TAI_NEU_XX.wav
sr=22050
Length of audio: 3.47015873015873
(128, 348)
1004_TAI_SAD_XX.wav
sr=22050
Length of audio: 3.436780045351474
(128, 345)
1004_TIE_ANG_XX.wav
sr=22050
Length of audio: 2.836190476190476
(128, 285)
1004_TIE_DIS_XX.wav
sr=22050
Length of audio: 4.337732426303855
(128, 435)
1004_TIE_FEA_XX.wav
sr=22050
Length of audio: 2.769478458049887
(128, 278)
1004_TIE_HAP_XX.wav
sr=22050
Length of a

1006_WSI_SAD_XX.wav
sr=22050
Length of audio: 3.9039002267573695
(128, 392)
1007_DFA_ANG_XX.wav
sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1007_DFA_DIS_XX.wav
sr=22050
Length of audio: 2.8695238095238094
(128, 288)
1007_DFA_FEA_XX.wav
sr=22050
Length of audio: 2.73609977324263
(128, 275)
1007_DFA_HAP_XX.wav
sr=22050
Length of audio: 2.2022222222222223
(128, 221)
1007_DFA_NEU_XX.wav
sr=22050
Length of audio: 2.635963718820862
(128, 265)
1007_DFA_SAD_XX.wav
sr=22050
Length of audio: 2.168843537414966
(128, 218)
1007_IEO_ANG_HI.wav
sr=22050
Length of audio: 2.43578231292517
(128, 245)
1007_IEO_ANG_LO.wav
sr=22050
Length of audio: 1.8351927437641724
(128, 184)
1007_IEO_ANG_MD.wav
sr=22050
Length of audio: 2.702721088435374
(128, 271)
1007_IEO_DIS_HI.wav
sr=22050
Length of audio: 2.2689795918367346
(128, 228)
1007_IEO_DIS_LO.wav
sr=22050
Length of audio: 1.9352834467120181
(128, 194)
1007_IEO_DIS_MD.wav
sr=22050
Length of audio: 2.035419501133787
(128, 205)
1007_IEO_FEA_HI.wav


sr=22050
Length of audio: 2.43578231292517
(128, 245)
1008_IOM_SAD_XX.wav
sr=22050
Length of audio: 3.703718820861678
(128, 372)
1008_ITH_ANG_XX.wav
sr=22050
Length of audio: 4.237596371882086
(128, 425)
1008_ITH_DIS_XX.wav
sr=22050
Length of audio: 3.803854875283447
(128, 382)
1008_ITH_FEA_XX.wav
sr=22050
Length of audio: 4.1374603174603175
(128, 415)
1008_ITH_HAP_XX.wav
sr=22050
Length of audio: 2.2356009070294784
(128, 225)
1008_ITH_NEU_XX.wav
sr=22050
Length of audio: 3.1031292517006803
(128, 312)
1008_ITH_SAD_XX.wav
sr=22050
Length of audio: 4.3710657596371885
(128, 439)
1008_ITS_ANG_XX.wav
sr=22050
Length of audio: 3.1698412698412697
(128, 318)
1008_ITS_DIS_XX.wav
sr=22050
Length of audio: 3.1364625850340135
(128, 315)
1008_ITS_FEA_XX.wav
sr=22050
Length of audio: 3.1698412698412697
(128, 318)
1008_ITS_HAP_XX.wav
sr=22050
Length of audio: 2.669342403628118
(128, 268)
1008_ITS_NEU_XX.wav
sr=22050
Length of audio: 3.1698412698412697
(128, 318)
1008_ITS_SAD_XX.wav
sr=22050
Length of

...

sr=22050
Length of audio: 2.635963718820862
(128, 265)
1078_ITH_NEU_XX.wav
sr=22050
Length of audio: 2.3356916099773244
(128, 235)
1078_ITH_SAD_XX.wav
sr=22050
Length of audio: 2.5359183673469388
(128, 255)
1078_ITS_ANG_XX.wav
sr=22050
Length of audio: 2.702721088435374
(128, 271)
1078_ITS_DIS_XX.wav
sr=22050
Length of audio: 2.4024036281179137
(128, 241)
1078_ITS_FEA_XX.wav
sr=22050
Length of audio: 2.769478458049887
(128, 278)
1078_ITS_HAP_XX.wav
sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1078_ITS_NEU_XX.wav
sr=22050
Length of audio: 2.1021315192743764
(128, 211)
1078_ITS_SAD_XX.wav
sr=22050
Length of audio: 2.4691609977324265
(128, 248)
1078_IWL_ANG_XX.wav
sr=22050
Length of audio: 2.936281179138322
(128, 295)
1078_IWL_DIS_XX.wav
sr=22050
Length of audio: 2.5359183673469388
(128, 255)
1078_IWL_FEA_XX.wav
sr=22050
Length of audio: 2.06875283446712
(128, 208)
1078_IWL_HAP_XX.wav
sr=22050
Length of audio: 2.6026303854875286
(128, 261)
1078_IWL_NEU_XX.wav
sr=22050
Length of

sr=22050
Length of audio: 2.836190476190476
(128, 285)
1079_TAI_DIS_XX.wav
sr=22050
Length of audio: 2.902902494331066
(128, 291)
1079_TAI_FEA_XX.wav
sr=22050
Length of audio: 2.2022222222222223
(128, 221)
1079_TAI_HAP_XX.wav
sr=22050
Length of audio: 2.3690702947845805
(128, 238)
1079_TAI_NEU_XX.wav
sr=22050
Length of audio: 2.6026303854875286
(128, 261)
1079_TAI_SAD_XX.wav
sr=22050
Length of audio: 2.4024036281179137
(128, 241)
1079_TIE_ANG_XX.wav
sr=22050
Length of audio: 3.3700680272108845
(128, 338)
1079_TIE_DIS_XX.wav
sr=22050
Length of audio: 3.3033560090702947
(128, 332)
1079_TIE_FEA_XX.wav
sr=22050
Length of audio: 2.669342403628118
(128, 268)
1079_TIE_HAP_XX.wav
sr=22050
Length of audio: 2.4691609977324265
(128, 248)
1079_TIE_NEU_XX.wav
sr=22050
Length of audio: 2.802857142857143
(128, 281)
1079_TIE_SAD_XX.wav
sr=22050
Length of audio: 3.1698412698412697
(128, 318)
1079_TSI_ANG_XX.wav
sr=22050
Length of audio: 2.702721088435374
(128, 271)
1079_TSI_DIS_XX.wav
sr=22050
Length o

sr=22050
Length of audio: 2.2756009070294785
(128, 229)
1084_DFA_NEU_XX.wav
sr=22050
Length of audio: 1.9352834467120181
(128, 194)
1084_DFA_SAD_XX.wav
sr=22050
Length of audio: 2.3690249433106576
(128, 238)
1084_IEO_ANG_HI.wav
sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1084_IEO_ANG_LO.wav
sr=22050
Length of audio: 2.3356916099773244
(128, 235)
1084_IEO_ANG_MD.wav
sr=22050
Length of audio: 2.4691609977324265
(128, 248)
1084_IEO_DIS_HI.wav
sr=22050
Length of audio: 2.6026303854875286
(128, 261)
1084_IEO_DIS_LO.wav
sr=22050
Length of audio: 2.2756009070294785
(128, 229)
1084_IEO_DIS_MD.wav
sr=22050
Length of audio: 2.6026303854875286
(128, 261)
1084_IEO_FEA_HI.wav
sr=22050
Length of audio: 2.2356009070294784
(128, 225)
1084_IEO_FEA_LO.wav
sr=22050
Length of audio: 2.06875283446712
(128, 208)
1084_IEO_FEA_MD.wav
sr=22050
Length of audio: 2.5359183673469388
(128, 255)
1084_IEO_HAP_HI.wav
sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1084_IEO_HAP_LO.wav
sr=22050
Lengt

sr=22050
Length of audio: 2.3690249433106576
(128, 238)
1089_ITH_HAP_XX.wav
sr=22050
Length of audio: 2.168843537414966
(128, 218)
1089_ITH_NEU_XX.wav
sr=22050
Length of audio: 2.035419501133787
(128, 205)
1089_ITH_SAD_XX.wav
sr=22050
Length of audio: 2.569251700680272
(128, 258)
1089_ITS_ANG_XX.wav
sr=22050
Length of audio: 2.769478458049887
(128, 278)
1089_ITS_DIS_XX.wav
sr=22050
Length of audio: 2.902902494331066
(128, 291)
1089_ITS_FEA_XX.wav
sr=22050
Length of audio: 3.1364625850340135
(128, 315)
1089_ITS_HAP_XX.wav
sr=22050
Length of audio: 2.635963718820862
(128, 265)
1089_ITS_NEU_XX.wav
sr=22050
Length of audio: 2.06875283446712
(128, 208)
1089_ITS_SAD_XX.wav
sr=22050
Length of audio: 2.769478458049887
(128, 278)
1089_IWL_ANG_XX.wav
sr=22050
Length of audio: 3.0030385487528344
(128, 301)
1089_IWL_DIS_XX.wav
sr=22050
Length of audio: 3.737097505668934
(128, 375)
1089_IWL_FEA_XX.wav
sr=22050
Length of audio: 2.769478458049887
(128, 278)
1089_IWL_HAP_XX.wav
sr=22050
Length of audi

sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1091_TAI_ANG_XX.wav
sr=22050
Length of audio: 2.3356916099773244
(128, 235)
1091_TAI_DIS_XX.wav
sr=22050
Length of audio: 2.669342403628118
(128, 268)
1091_TAI_FEA_XX.wav
sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1091_TAI_HAP_XX.wav
sr=22050
Length of audio: 2.3356916099773244
(128, 235)
1091_TAI_NEU_XX.wav
sr=22050
Length of audio: 2.2689795918367346
(128, 228)
1091_TAI_SAD_XX.wav
sr=22050
Length of audio: 2.168843537414966
(128, 218)
1091_TIE_ANG_XX.wav
sr=22050
Length of audio: 2.669342403628118
(128, 268)
1091_TIE_DIS_XX.wav
sr=22050
Length of audio: 3.7704761904761903
(128, 378)
1091_TIE_FEA_XX.wav
sr=22050
Length of audio: 2.5025396825396826
(128, 251)
1091_TIE_HAP_XX.wav
sr=22050
Length of audio: 2.836190476190476
(128, 285)
1091_TIE_NEU_XX.wav
sr=22050
Length of audio: 2.2756009070294785
(128, 229)
1091_TIE_SAD_XX.wav
sr=22050
Length of audio: 2.2356009070294784
(128, 225)
1091_TSI_ANG_XX.wav
sr=22050
Length 